### Zadanie 3
W tym zadaniu mieliśmy napisać schematy iteracji wg metody Newtona dla równań nieliniowych: $$ \begin{aligned}
(a) \quad & x^3 - 2x - 5 = 0 \\
(b) \quad & e^{-x} = x \\
(c) \quad & x \sin(x) = 1.
\end{aligned} $$ Następnie trzeba było ustalić ile iteracji należy wykonać, aby osiągnąć 24 i 53-bitową dokładność wyniku, jeśli początkowe przybliżenie pierwiastka $ x_0 $ ma dokładność 4 bitów.

In [65]:
import numpy as np
import pandas as pd

Na początku zdefiniowaliśmy funkcje $ f_1(x) = x^3 - 2x - 5 $, $ f_2(x) = e^{-x} - x $, $ f_3(x) = xsin(x) - 1 $ oraz ich pochodne. Następnie stworzyliśmy schematy iteracyjne metody Newtona dla każdej z funkcji, zgodnie ze wzorem: $$ x_{k+1} = x_k - \frac{f(x_k)}{f'(x_k)} $$ Za pomocą tych schematów wyznaczyliśmy najpierw pierwiastki z dokładnością 4 bitów. W celu ustalenia dokładności porównywaliśmy różnicę wartości $ x $ w kolejnych iteracjach z $ \epsilon=2^{-4} $:

In [66]:
fa = lambda x: x**3 - 2*x - 5
dfa = lambda x: 3*x**2 - 2

fb = lambda x: np.exp(-x) - x
dfb = lambda x: -np.exp(-x) - 1

fc = lambda x: x * np.sin(x) - 1
dfc = lambda x: np.sin(x) + x * np.cos(x)

In [67]:
iter_a = lambda x: x - fa(x) / dfa(x)
iter_b = lambda x: x - fb(x) / dfb(x)
iter_c = lambda x: x - fc(x) / dfc(x)

def solve_iter_with_precision(iter_fun, x0, tolerance=1e-7, max_iter=1000):
    x_prev = x0
    x_curr = iter_fun(x_prev)
    iteration = 1
    while abs(x_curr - x_prev) > tolerance and iteration < max_iter:
        x_prev = x_curr
        x_curr = iter_fun(x_prev)
        iteration += 1
    return x_curr, iteration

solutions4bit = [solve_iter_with_precision(iter_a, np.double(2), np.pow(1/2, 4))[0], 
    solve_iter_with_precision(iter_b, np.double(0.5), np.pow(1/2, 4))[0],
    solve_iter_with_precision(iter_c, np.double(1), np.pow(1/2, 4))[0]]
pd.DataFrame(solutions4bit, ["f1", "f2", "f3"], columns=['x0 4bit'])

,x0 4bit
f1,2.094568
f2,0.567143
f3,1.114157


Następnie wyznaczyliśmy przybliżenia z 24-bitową dokładnością:

In [68]:
solutions24bit = pd.DataFrame(
    [solve_iter_with_precision(iter_a, solutions4bit[0], np.pow(1/2, 24)), 
    solve_iter_with_precision(iter_b, solutions4bit[1], np.pow(1/2, 24)),
    solve_iter_with_precision(iter_c, solutions4bit[2], np.pow(1/2, 24))],
    ["f1", "f2", "f3"], columns=['x0 24bit', 'Liczba iteracji']
)

solutions24bit

,x0 24bit,Liczba iteracji
f1,2.094551,2
f2,0.567143,2
f3,1.114157,1


Jak widać w zamieszczonej powyżej tabelce, schematy iteracyjne Newtona bardzo szybko otrzymują wynik z wymaganą dokładnością, co pokazuje skuteczność tej metody.

Potem wyznaczyliśmy przybliżenia z 53-bitową dokładnością: 

In [69]:
solutions53bit = pd.DataFrame(
    [solve_iter_with_precision(iter_a, solutions4bit[0], np.pow(1/2, 53)), 
    solve_iter_with_precision(iter_b, solutions4bit[1], np.pow(1/2, 53)),
    solve_iter_with_precision(iter_c, solutions4bit[2], np.pow(1/2, 53))],
    ["f1", "f2", "f3"], columns=['x0 53bit', 'Liczba iteracji']
)

solutions53bit

,x0 53bit,Liczba iteracji
f1,2.094551,3
f2,0.567143,3
f3,1.114157,1000


W tym przypadku liczba iteracji jest wciąż bardzo niska dla funkcji $ f1 $ i $ f2 $, ale w przypadku $ f3 $ nie udało się uzyskać wymaganej dokładności. Funkcja ta prawdopodobnie nie spełnia warunków zbieżności dla metody Newtona.